In [52]:
from CheckEnv import check_env
from CheckPlayer import check_agent
from setup import SHOT_PATH, time_run_game
import pandas as pd
import importlib.util
import numpy as np
import sys, time, shutil
import json

SHOT_PATH = 'A:/AutoTrain/GAME/'

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHOT_PATH}base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

def load_module_player(player):
    return  importlib.util.spec_from_file_location('Agent_player', f"{SHOT_PATH}Agent/{player}/Agent_player.py").loader.load_module()

In [ ]:
df_run = pd.read_json(f'{SHOT_PATH}Log/State.json')
df_agent = pd.read_json(f'{SHOT_PATH}Log/StateAgent.json')
df_env = pd.read_json(f'{SHOT_PATH}Log/StateEnv.json')

list_tong = list(df_run.index)
dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
dict_agent = json.load(open(f'{SHOT_PATH}Log/agent_all.json'))
PASS_LEVEL = 250
df_run

In [55]:
TEXT_ADD = """
import numpy as np
from numba import njit
import sys, os
from setup import SHOT_PATH
import importlib.util
game_name = sys.argv[1]

def setup_game(game_name):
    spec = importlib.util.spec_from_file_location('env', f"{SHOT_PATH}base/{game_name}/env.py")
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module 
    spec.loader.exec_module(module)
    return module

env = setup_game(game_name)

getActionSize = env.getActionSize
getStateSize = env.getStateSize
getAgentSize = env.getAgentSize

getValidActions = env.getValidActions
getReward = env.getReward

normal_main = env.normal_main
numba_main_2 = env.numba_main_2
"""

def add_text_to_file(agent_name):
    shutil.move(f'{SHOT_PATH}Agent/{agent_name}/Agent_player.py', f'{SHOT_PATH}Agent/{agent_name}/Agent_player.txt')
    time.sleep(0.5)
    with open(f'{SHOT_PATH}Agent/{agent_name}/Agent_player.txt', 'r') as original: data = original.read()
    time.sleep(0.5)
    with open(f'{SHOT_PATH}Agent/{agent_name}/Agent_player.txt', 'w') as modified: modified.write(f"{TEXT_ADD}\n" + data)
    time.sleep(0.5)
    shutil.move(f'{SHOT_PATH}Agent/{agent_name}/Agent_player.txt', f'{SHOT_PATH}Agent/{agent_name}/Agent_player.py')
    time.sleep(0.5)

add_text_to_file('agent_test')

In [ ]:
dict_level

In [4]:
df_run.drop(columns=['system0_1672720455']).to_json(f'{SHOT_PATH}Log/State.json')

In [14]:
def save_json(path_save_json, dict_save):
    with open(path_save_json, 'w') as f:
        json.dump(dict_save, f, indent= 4)
save_json(f'{SHOT_PATH}Log/level_game.json', dict_level)

In [50]:
def choose_game_level_train(agent_name):
    dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
    dict_agent = json.load(open(f'{SHOT_PATH}Log/agent_all.json'))

    if len(dict_level) > 0: #Nếu số game lớn hơn 0
        env_name = list(dict_level.keys())[0]

        if dict_agent[agent_name]['First train'] == False: #Khi chưa train và chọn level xong cho game đầu tiên
            state_train_env = dict_agent[agent_name][env_name]['State_level']
            if state_train_env[0] == 0: #Là chưa được train
                level_train = state_train_env[1] #Báo hiệu level sẽ được train tiếp theo
                if level_train == 0:
                    level_train = dict_level[env_name]['level_max'] - 2
                dict_agent[agent_name][env_name]['State_level'][0] = 1 #Gán để báo hiệu đang được train
                dict_agent[agent_name][env_name]['State_level'][1] = level_train
                save_json(f'{SHOT_PATH}Log/agent_all.json', dict_agent)
                return env_name, level_train

    for env_name in dict_agent[agent_name]:
        if env_name not in ['Elo', 'First train', 'Level Train', 'Agent Save']:
            state_train_env = dict_agent[agent_name][env_name]['State_level']
            if state_train_env[0] == 0:
                if dict_agent[agent_name]['Agent Save'] == True: #Nếu là agent đang mạnh thì train tiếp level 
                    level_train = state_train_env[1]
                    dict_agent[agent_name][env_name]['State_level'][0] = 1
                else:
                    level_train = state_train_env[1]//2
                    dict_agent[agent_name][env_name]['State_level'][0] = -1 #Gán cái này agent này yếu nên chỉ train mỗi game 1 level thôi
                if level_train == 0:
                    level_train = dict_agent[agent_name]['Level Train']
                dict_agent[agent_name][env_name]['State_level'][1] = level_train
                save_json(f'{SHOT_PATH}Log/agent_all.json', dict_agent)
                return env_name, level_train
    return False, False


choose_game_level_train('system0_1672806467')

('system0_1672720433', 4)

In [70]:
def after_train(current_lv, env_name, agent_name, win, condition_pass_level):
    dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
    dict_agent = json.load(open(f'{SHOT_PATH}Log/agent_all.json'))

    state_train_env = dict_agent[agent_name][env_name]['State_level']

    if dict_agent[agent_name]['First train'] == False:
        if current_lv == dict_level[env_name]['level_max']:
            dict_agent[agent_name]['First train'] = True
            dict_agent[agent_name]['Level Train'] = current_lv
            dict_agent[agent_name][env_name]['State_level'][0] = -1
        
        elif current_lv < dict_level[env_name]['level_max'] - 2:
            dict_agent[agent_name]['First train'] = True
            dict_agent[agent_name]['Level Train'] = current_lv
            dict_agent[agent_name][env_name]['State_level'][0] = -1
            dict_agent[agent_name]['Agent Save'] == False
        else:
            if win <= condition_pass_level:
                if current_lv == dict_level[env_name]['level_max'] -2:
                    dict_agent[agent_name]['First train'] = False
                    dict_agent[agent_name][env_name]['State_level'][0] = 0
                    dict_agent[agent_name][env_name]['State_level'][1] = current_lv//2
                    dict_agent[agent_name]['Agent Save'] == False
                else:
                    dict_agent[agent_name]['First train'] = True
                    dict_agent[agent_name]['Level Train'] = current_lv
                    dict_agent[agent_name][env_name]['State_level'][0] = -1
                    dict_agent[agent_name][env_name]['State_level'][1] = current_lv-1
            else:
                level_test = dict_level[env_name]['level_max']

                __env__ = setup_game(env_name)
                data_agent_env = list(np.load(f'{SHOT_PATH}Agent/{agent_name}/Data/{env_name}_{current_lv}/Train.npy',allow_pickle=True))
                add_game_to_syspath(env_name)
                p0 = load_module_player(agent_name).Agent
                win_test, per = __env__.numba_main_2(p0, 10000, data_agent_env, level_test)
                
                dict_agent = json.load(open(f'{SHOT_PATH}Log/agent_all.json'))
                if win_test > condition_pass_level:
                    dict_agent[agent_name][env_name]['State_level'][1] = level_test
                else:
                    dict_agent[agent_name][env_name]['State_level'][1] = current_lv + 1

                dict_agent[agent_name]['Level Train'] = current_lv
                dict_agent[agent_name][env_name]['State_level'][0] = 0

    else:
        dict_agent[agent_name][env_name]['State_level'][0] = -1
        dict_agent[agent_name]['Agent Save'] == False

    save_json(f'{SHOT_PATH}Log/level_game.json', dict_level)
    save_json(f'{SHOT_PATH}Log/agent_all.json', dict_agent)
        
    # return dict_agent

after_train(6, 'system0_1672717758', 'system0_1672806467', 1, 0)

FileNotFoundError: [Errno 2] No such file or directory: 'A:/AutoTrain/GAME/Agent/system0_1672806467/Data/system0_1672717758_6/Train.npy'

In [ ]:
def add_to_strong_agent(env_name, agent_name, level, win, condition_pass_level):

    if level == dict_level[env_name]['level_max'] and win > condition_pass_level: # (10000/(__env__.getAgentSize()))
        check_dk = True
        for env_name_check in dict_agent[agent_name]:
            if env_name_check not in ['Elo', 'First train', 'Level Train']:
                check_condition_1_env = False

                level_max = dict_level[env_name_check]['level_max']
                for id_level in range(level_max, level_max-3, -1):
                    if str(id_level) in dict_agent[agent_name][env_name_check]:
                        if len(dict_agent[agent_name][env_name_check][str(id_level)]) > 0:
                            if dict_agent[agent_name][env_name_check][str(id_level)][0] > condition_pass_level:
                                check_condition_1_env = True
                                break

                if check_condition_1_env == False:
                    check_dk = False
            
        df_run = pd.read_json(f'{SHOT_PATH}Log/State.json')
        if check_dk == True: 
            if agent_name not in list(df_run["ID"]):
                if os.path.exists(f'{SHOT_PATH}Agent/{agent_name}/Data/{env_name}_0/') == False:
                    folder_name_old = f'{SHOT_PATH}Agent/{agent_name}/Data/{env_name}_{level}/'
                    folder_name_new = f'{SHOT_PATH}Agent/{agent_name}/Data/{env_name}_0/'
                    os.rename(folder_name_old, folder_name_new)

                id_add_agent = len(df_run)
                df_run.loc[id_add_agent] = 0
                df_run['ID'][id_add_agent] = agent_name
                df_run[env_name][id_add_agent] = win//10
                df_run.to_json(f'{SHOT_PATH}Log/State.json')

In [21]:
def update_json():
    dict_level = json.load(open(f'{SHOT_PATH}Log/level_game.json'))
    dict_agent = json.load(open(f'{SHOT_PATH}Log/agent_all.json'))

    for agent in dict_agent:
        for env_name in dict_level:
            if env_name not in dict_agent[agent]:
                dict_agent[agent][env_name] = {'State_level':[0, 0 , dict_level[env_name]['level_max']],
                                                'Agent Save': True}
            for level in range(dict_level[env_name]['level_max']+1):
                if str(level) not in dict_agent[agent][env_name]:
                    dict_agent[agent][env_name][level] = []
    save_json(f'{SHOT_PATH}Log/agent_all.json', dict_agent)


update_json()